# Waveform user interface notebook
---
Click the Kernel tab above to start the notebook:

    Kernel --> Restart & Run All

Choose the raw data file you want to look at, as well as the waveform.

In [1]:
from scipy.ndimage import gaussian_filter
from scipy.signal import butter, filtfilt, sosfilt
import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import uproot
import glob
import os
from collections import OrderedDict
import traitlets
import ipywidgets as ipywid
from ipywidgets import interact, widgets, Select, Layout, Button, interactive_output, HBox, VBox

from IPython.display import HTML, display, Javascript

display(HTML('''<style>
    .widget-label { min-width: 30ex !important; }
    .widget { min-width: 40ex !important; }
</style>'''))
from platform import python_version


Necessary libraries and packages imported.

In [2]:
# main directory for data:
datahome = "/p/lustre1/nexouser/data/StanfordData/"

# go through run numbers:
def get_run_list(person):
    direct = datahome + person
    if os.path.isdir(direct):
        new_list = os.listdir(direct)
    else:
        print("Warning " + direct + " does not exist")
    new_list.sort()
    return new_list

def upgrade_run_widget(*args):
    run_widget.options = get_run_list(person_widget.value)
    
def get_runtype_list(person, run):
    direct = datahome + person + "/" + run + "/"
    if os.path.isdir(direct):
        new_list = os.listdir(direct)
    else:
        print("Warning " + direct + " does not exist")
    new_list.sort()
    return new_list
    
def upgrade_runtype_widget(*args):
    runtype_widget.options = get_runtype_list(person_widget.value, run_widget.value)
    
def get_file_list(person, run, runtype):
    direct = datahome + person + "/" + run + "/" + runtype + "/raw_data/"
    if os.path.isdir(direct):
        new_list = os.listdir(direct)
    else:
        print("Warning " + direct + " does not exist")
        new_list = []
    new_list = [i for i in new_list if "root" in i]
    new_list.sort()
    return new_list

def upgrade_file_widget(*args):
    file_widget.options = get_file_list(person_widget.value, run_widget.value, runtype_widget.value)
    
person_options = os.listdir(datahome)
person = 'jacopo'
person_options.sort()
run_options = os.listdir(datahome+person)
# remove non directories 
run_options = [s for s in run_options if "py" not in s]
run_options  = [s for s in run_options if "PulserTest" not in s]
default_file = "tier1_SIS3316Raw_20221006230202DS06_Run38_SiPM30p0_hv840V_1-ngm.root"

person_widget = Select(options=person_options, value="jacopo")
run_widget = Select(options=get_run_list("jacopo"), value="38th", continuous_update=False)
runtype_widget = Select(options=get_runtype_list("jacopo", "38th"), value="DS06_Xe127inj", continuous_update=True)
file_item_layout = Layout(width='auto')
file_widget = Select(options=get_file_list("jacopo", "38th", "DS06_Xe127inj"), value=default_file, continuous_update=True, layout=file_item_layout)


person_widget.observe(upgrade_run_widget, 'value')
run_widget.observe(upgrade_runtype_widget, 'value')
runtype_widget.observe(upgrade_file_widget, 'value')
file_widget.observe(upgrade_file_widget)

In [3]:
display(person_widget)
display(run_widget)
display(runtype_widget)
display(file_widget)

Select(index=1, options=('angelico', 'jacopo', 'vidal4'), value='jacopo')

Select(options=('38th', 'Run37'), value='38th')

Select(index=6, options=('DS00', 'DS01_3p5kg', 'DS02_6p0kg', 'DS03_7p8kg', 'DS04_prerecirc_increase', 'DS05_hi…

Select(index=187, layout=Layout(width='auto'), options=('tier1_SIS3316Raw_20221006222639DS06_Run38_SiPM30p0_hv…

In [8]:
# have to have that function otherwise the object type is none and is not callable and so error.
def run(ev):
    display(Javascript('IPython.notebook.execute_cells_below()'))
    
button_file_in = widgets.Button(description='Load chosen file')
button_file_in.on_click(run)
display(button_file_in)

<IPython.core.display.Javascript object>

Button(description='Load chosen file', style=ButtonStyle())

In [9]:
file_widget.observe(upgrade_file_widget)
datapath = datahome + person_widget.value + "/" + run_widget.value + "/" + runtype_widget.value + "/raw_data/" + file_widget.value

print('Loaded file:')
datapath_widget = widgets.Text(description='', value=datapath, layout={'width': '750px'})
display(datapath_widget)
#print("loaded file:", datapath_widget.value)
flist = datapath_widget.value

if type(flist) == list:
    i_file = 1
    l_file = 3
    wfm = uproot.concatenate([select_files+':HitTree/HitTree/_waveform' for select_files in flist[i_file:l_file]])['_waveform']
    slot = uproot.concatenate([select_files+':HitTree/HitTree/_slot' for select_files in flist[i_file:l_file]])['_slot']
    channel = uproot.concatenate([select_files+':HitTree/HitTree/_channel' for select_files in flist[i_file:l_file]])['_channel']
    timestamp = uproot.concatenate([select_files+':HitTree/HitTree/_rawclock' for select_files in flist[i_file:l_file]])['_rawclock']
else:
    wfm = uproot.open(flist+':HitTree/HitTree/_waveform').array()
    slot = uproot.open(flist+':HitTree/HitTree/_slot').array()
    channel = uproot.open(flist+':HitTree/HitTree/_channel').array()
    timestamp = uproot.open(flist+':HitTree/HitTree/_rawclock').array()

Loaded file:


Text(value='/p/lustre1/nexouser/data/StanfordData/jacopo/38th/DS17_After_Fast_Recirc/raw_data/tier1_SIS3316Raw…

The first two lines represent general quantities. Choose the event number you want to look at and the sampling frequency. Concerning the number of boards and number of channel per board, the default for Stanford are the numbers pre-entered.
Following the first two lines, column 1 controls SiPM quantities and options, and column 2 controls the charge quantities and options.

In [10]:
import awkward as ak

def DecayTimeCorrection(input_wfm, decay_time_ns, sampling_period_ns):
    # code from Waveform.py, should import from there
    # Here I'll assume the decay time is in units of mircoseconds
    # and the sampling period is in units of ns
    input_wfm = ak.to_numpy(input_wfm)
    new_wfm = np.copy( input_wfm )
    decay_time_us = decay_time_ns*1e-3
    for i in range(len(input_wfm)-1):
        new_wfm[i+1] = new_wfm[i] - np.exp( - (sampling_period_ns/1.e3) / decay_time_us ) * input_wfm[i] + input_wfm[i+1]
    return new_wfm

def lowpass_filter(data, cutoff, sampling_freq, order, nyq):
    norm_cutoff = cutoff/nyq
    # filter coeff.
    a, b = butter(order, norm_cutoff, btype='low', analog=False)
    y = filtfilt(a, b, data)
    return y

def Power_spectral_density(t, f):
    # look at spectral density power:
    n = len(t)
    f_hat = np.fft.fft(f, n)
    PSD = f_hat * np.conj(f_hat)/n
    dt= 1/(sampling_freq)  # samples/s
    freq = (1/(dt*n))*np.arange(n)
    R = np.arange(1, np.floor(n/2), dtype='int')
    return R, freq, PSD

def parameters(Check_SiPM, Event_number, sampl_freq, nb_boards, nb_ch_board, 
               waveform_nb, smoothing_sipm, Check_Charge, waveform_nb_charge, 
               Check_all_charge, Check_all_sipm, smoothing_charge, PSD_sipm, PSD_charge, 
               Check_baseline_sipm, Check_baseline_charge, Decay_sipm, Decay_charge, 
               Check_Gaussian_filter_sipm, Check_Gaussian_filter_charge, Check_Processed_pulse_charge):
    
    software_ch = np.add(slot*16,channel)
    n_channels = nb_boards*nb_ch_board
    event_nb = Event_number
    j = event_nb*48
    Decay_charge_mus = Decay_charge*1e-3
    
    # check if we want to look at SiPM pulses
    if Check_SiPM==True:
        list_channels = []
        for j_i in range(j, j+n_channels):
            if software_ch[j_i]<16:
                list_channels.append(j_i)

        plt.rc('axes', labelsize=12)
        plt.rcParams['font.size'] = 10
        array_channel = np.array(list_channels).reshape(4, 4)
        
        if Check_all_sipm == True:
            fig, ax = plt.subplots(4, 4, figsize=[17, 18])
            for l in range(0, 4):
                for c in range(0, 4):
                    ch = array_channel[l][c]
                    ax[l, c].plot(np.arange(len(wfm[ch]))/sampling_freq*1e6,wfm[ch]-np.mean(wfm[ch,:200])+400*software_ch[ch], label=str(l*4+c))
                    ax[l, c].set_xlabel('Time [$\mu$s]')
                    ax[l, c].legend()
            fig.suptitle("SiPM pulses", fontsize=20)
            plt.show()
        else:
            for l in range(0, 4):
                for c in range(0, 4):
                    ch = array_channel[l][c]


        if 0 <= waveform_nb and waveform_nb < 4:
            c1 = 0
            c2 = waveform_nb
        elif 4 <= waveform_nb and waveform_nb < 8:
            c1 = 1
            c2 = waveform_nb-4
        elif 8 <= waveform_nb and waveform_nb < 12:
            c1 = 2
            c2 = waveform_nb-8
        elif 12 <= waveform_nb and waveform_nb < 16:
            c1 = 3
            c2 = waveform_nb-12
        elif waveform_nb < 0 or waveform_nb >17:
            print("Invalid pulse number")

        
        plt.figure(figsize=(15, 6))
        t = np.arange(len(wfm[array_channel[c1][c2]]))/sampling_freq*1e6
        f = wfm[array_channel[c1][c2]]-np.mean(wfm[array_channel[c1][c2],:200])+400*software_ch[array_channel[c1][c2]]
        
        t_baseline = t[t<30]
        index_end_BL = len(t_baseline)
        f_baseline = np.full(index_end_BL, np.mean(f[:index_end_BL]))
        
        if Check_baseline_sipm == False:
            plt.plot(t, f, label='Raw pulse')
            nyq = 0.5*sampling_freq
            if Check_Gaussian_filter_sipm==True:
                f_filtered = gaussian_filter(f, smoothing_sipm/Decay_sipm)
                plt.plot(t, f_filtered, label='Gaussian filter')
            plt.plot(t_baseline, f_baseline, 'r', label='Baseline')
            plt.grid()
            plt.legend()
            plt.title("Selected SiPM pulse")
            plt.xlabel(r"Time [$\mu$s]")
        else:
            f_remove_baseline = np.full(len(t), np.mean(f[:index_end_BL]))
            plt.plot(t, f-f_remove_baseline, label='Raw pulse')
            nyq = 0.5*sampling_freq
            if Check_Gaussian_filter_sipm==True:
                f_filtered = gaussian_filter(f-f_remove_baseline, smoothing_sipm/Decay_sipm)
                plt.plot(t, f_filtered, label='Gaussian filter')
            plt.plot(t_baseline, f_baseline-f_baseline, 'r', label='Baseline')
            plt.grid()
            plt.legend()
            plt.title("Selected SiPM pulse")
            plt.xlabel(r"Time [$\mu$s]")
        
    
        if PSD_sipm==True:
            # look at spectral density power:
            R, freq, PSD = Power_spectral_density(t, f)
            plt.figure(figsize=(15, 6))
            plt.plot(freq[R]*1e-6, PSD[R])
            plt.xlabel("frequency [MHz]")
            plt.ylabel(r"Spectral density power [$V^2$/Hz]")
            plt.xlim(0.2, 12.5)
            plt.xscale("log")
            plt.show()
    
    # Check if we want to look at Charge pulses
    if Check_Charge==True:
        
        list_channels = []
        for j_i in range(j, j+n_channels):
            if software_ch[j_i]>=16:
                list_channels.append(j_i)
            elif software_ch[j_i]==45:
                continue

        plt.rc('axes', labelsize=12)
        plt.rcParams['font.size'] = 10
        
        array_channel = np.array(list_channels).reshape(8, 4)
        if Check_all_charge == True:
            fig, ax = plt.subplots(8, 4, figsize=[17, 37])
            for l in range(0, 8):
                for c in range(0, 4):
                    ch = array_channel[l][c]
                    ax[l, c].plot(np.arange(len(wfm[ch]))/sampling_freq*1e6,wfm[ch]-np.mean(wfm[ch,:200]), label=str(l*4+c))
                    ax[l, c].set_xlabel('Time [$\mu$s]')
                    ax[l, c].legend()
            plt.suptitle("Charge pulses", fontsize=20)
            plt.show()
        else:
            for l in range(0, 8):
                for c in range(0, 4):
                    ch = array_channel[l][c]
        
        if 0 <= waveform_nb_charge and waveform_nb_charge < 4:
            c1 = 0
            c2 = waveform_nb_charge
        elif 4 <= waveform_nb_charge and waveform_nb_charge < 8:
            c1 = 1
            c2 = waveform_nb_charge-4
        elif 8 <= waveform_nb_charge and waveform_nb_charge < 12:
            c1 = 2
            c2 = waveform_nb_charge-8
        elif 12 <= waveform_nb_charge and waveform_nb_charge < 16:
            c1 = 3
            c2 = waveform_nb_charge-12
        elif 16<= waveform_nb_charge and waveform_nb_charge < 20:
            c1 = 4
            c2 = waveform_nb_charge-16
        elif 20 <= waveform_nb_charge and waveform_nb_charge < 24:
            c1 = 5
            c2 = waveform_nb_charge-20
        elif 24 <= waveform_nb_charge and waveform_nb_charge < 28:
            c1 = 6
            c2 = waveform_nb_charge-24
        elif 28 <= waveform_nb_charge and waveform_nb_charge < 32:
            c1 = 7
            c2 = waveform_nb_charge-28
        elif waveform_nb_charge < 0 or waveform_nb_charge > 32:
            print("Invalid pulse number")
            
        
        plt.figure(figsize=(15, 6))
        t = np.arange(len(wfm[array_channel[c1][c2]]))/sampling_freq*1e6
        f = wfm[array_channel[c1][c2]]-np.mean(wfm[array_channel[c1][c2],:200])+400*software_ch[array_channel[c1][c2]]
        t_baseline = t[t<30]
        index_end_BL = len(t_baseline)
        f_baseline = np.full(index_end_BL, np.mean(f[:index_end_BL]))
        f_remove_baseline = np.full(len(t), np.mean(f[:index_end_BL]))
        
        if Check_baseline_charge == False and Check_Processed_pulse_charge==False:
            plt.plot(t, f, label='Raw pulse')
            nyq = 0.5*sampling_freq
            if Check_Gaussian_filter_charge==True:
                f_filtered = gaussian_filter(f, smoothing_charge/Decay_charge)
                plt.plot(t, f_filtered, label='Gaussian filter')
            plt.plot(t_baseline, f_baseline, 'r', label='Baseline')
            plt.grid()
            plt.legend()
            plt.title("Selected SiPM pulse")
            plt.xlabel(r"Time [$\mu$s]")
        elif Check_baseline_charge ==True and Check_Processed_pulse_charge==False:
            plt.plot(t, f-f_remove_baseline, label='Raw pulse')
            nyq = 0.5*sampling_freq
            if Check_Gaussian_filter_charge==True:
                f_filtered = gaussian_filter(f, smoothing_charge/Decay_charge)
                plt.plot(t, f_filtered, label='Gaussian filter')
            plt.plot(t_baseline, f_baseline-f_baseline, 'r', label='Baseline')
            plt.grid()
            plt.legend()
            plt.title("Selected SiPM pulse")
            plt.xlabel(r"Time [$\mu$s]")
        elif Check_Processed_pulse_charge==True:
            plt.plot(t, f-f_remove_baseline, label='Raw pulse')
            plt.plot(t_baseline, f_baseline-f_baseline, 'r', label='Baseline')
            sampl_period_ns = 1e9/sampl_freq
            f_filtered = gaussian_filter(f-f_remove_baseline, smoothing_charge/sampl_period_ns)
            #f_baseline_removed = f_filtered-np.full(len(t), np.mean(f[:index_end_BL]))
            f_corrected = DecayTimeCorrection(f_filtered, Decay_charge, sampl_period_ns)
            plt.plot(t, f_corrected,'k', label='Processed')
            plt.grid()
            plt.legend()
            plt.title("Selected charge pulse")
            plt.xlabel(r"Time [$\mu$s]")
        
        if PSD_charge==True:
            # look at spectral density power:
            R, freq, PSD = Power_spectral_density(t, f)
            plt.figure(figsize=(15, 6))
            plt.plot(freq[R]*1e-6, PSD[R])
            plt.xlabel("frequency [MHz]")
            plt.ylabel(r"Spectral density power [$V^2$/Hz]")
            plt.xlim(0.2, 12.5)
            plt.xscale("log")
            plt.show()
    
    return 

# default run parameters
sampling_freq = 25e6
n_boards = 3
n_ch_board = 16
pulse_channel = 13
gaussian_smooth_sipm = 80 # ns 
gaussian_smooth_charge = 500 # ns
order_input = 2
pulse_channel_charge = 15
decay_time_sipm = 200 # ns
decay_time_charge = 500e3 # ns

CheckBox_SiPM = widgets.Checkbox(description="SiPM pulses?", value=True, layout=widgets.Layout(width='100%'))
IntText_Event_nb = widgets.BoundedIntText(description='Event number:', value=0, min=0)
IntText_Sampl_freq = widgets.IntText(description="Sampling frequency (Hz):", value=sampling_freq)
IntText_Nb_boards = widgets.IntText(description="Number of boards:", value=n_boards)
IntText_Pulse_nb = widgets.IntText(description="Pulse number:", value=pulse_channel)
IntText_Nb_channel_board = widgets.IntText(description="Number of channel per board:", value = n_ch_board)
FloatText_Gaussian_smooth_sipm = widgets.FloatText(description="Gaussian smoothing (ns):", value=gaussian_smooth_sipm)
CheckBox_Charge = widgets.Checkbox(description="Charge pulses?", value=False, layout=widgets.Layout(width='100%'))
IntText_Pulse_nb_charge = widgets.IntText(description="Pulse number:", value=pulse_channel_charge)
CheckBox_all_charge_pulses = widgets.Checkbox(description="All charge pulses?", value=True, layout=widgets.Layout(width='100%'))
CheckBox_all_sipm_pulses = widgets.Checkbox(description="All SiPM pulses?", value=True, layout=widgets.Layout(width='100%'))
FloatText_Gaussian_smooth_charge = widgets.FloatText(description="Gaussian smoothing (ns):", value=gaussian_smooth_charge)
CheckBox_PSD_sipm = widgets.Checkbox(description="Power density spectrum?", value=False, layout=widgets.Layout(width='100%'))
CheckBox_PSD_charge = widgets.Checkbox(description="Power density spectrum?", value=False, layout=widgets.Layout(width='100%'))
CheckBox_Baseline_removed_sipm = widgets.Checkbox(description="Baseline removed (SiPM pulse)?", value=False, layout=widgets.Layout(width='100%'))
CheckBox_Baseline_removed_charge = widgets.Checkbox(description="Baseline removed (Charge pulse)", value=False, layout=widgets.Layout(width='100%'))
CheckBox_gaussian_filter_sipm = widgets.Checkbox(description="Gaussian filter for SiPM pulse?", value=False, layout=widgets.Layout(width='100%'))
CheckBox_gaussian_filter_charge = widgets.Checkbox(description="Gaussian filter for charge pulse?", value=False, layout=widgets.Layout(width='100%'))
FloatText_decaytime_sipm = widgets.FloatText(description="Decay time preamp SiPM (ns):", value=decay_time_sipm)
FloatText_decaytime_charge = widgets.FloatText(description="Decay time preamp charge (ns):", value=decay_time_charge)
CheckBox_Gaussian_filter_sipm = widgets.Checkbox(description="Gaussian filter?", value=False, layout=widgets.Layout(width='100%'))
CheckBox_Gaussian_filter_charge = widgets.Checkbox(description="Gaussian filter?", value=False, layout=widgets.Layout(width='100%'))
CheckBox_Processed_pulse_charge = widgets.Checkbox(description="Processed pulse?", value=False, layout=widgets.Layout(width='100%'))

output = interactive_output(parameters, {'Event_number': IntText_Event_nb, 
                                         'sampl_freq': IntText_Sampl_freq, 
                                         'nb_boards': IntText_Nb_boards, 
                                         'nb_ch_board': IntText_Nb_channel_board,
                                         'Check_SiPM': CheckBox_SiPM, 
                                         'waveform_nb': IntText_Pulse_nb, 
                                         'smoothing_sipm': FloatText_Gaussian_smooth_sipm, 
                                         'Check_Charge': CheckBox_Charge,
                                         'waveform_nb_charge': IntText_Pulse_nb_charge, 
                                         'Check_all_charge': CheckBox_all_charge_pulses, 
                                         'Check_all_sipm': CheckBox_all_sipm_pulses,
                                         'smoothing_charge': FloatText_Gaussian_smooth_charge,
                                         'PSD_sipm': CheckBox_PSD_sipm,
                                         'PSD_charge': CheckBox_PSD_charge,
                                         'Check_baseline_sipm': CheckBox_Baseline_removed_sipm,
                                         'Check_baseline_charge': CheckBox_Baseline_removed_charge,
                                         'Decay_sipm': FloatText_decaytime_sipm,
                                         'Decay_charge': FloatText_decaytime_charge,
                                         'Check_Gaussian_filter_sipm': CheckBox_Gaussian_filter_sipm, 
                                         'Check_Gaussian_filter_charge': CheckBox_Gaussian_filter_charge,
                                         'Check_Processed_pulse_charge': CheckBox_Processed_pulse_charge})#,
                                         #'select_filter': Select_filter_sipm})
# column1:
col1 = VBox([IntText_Event_nb, IntText_Sampl_freq, CheckBox_SiPM, CheckBox_all_sipm_pulses, 
             IntText_Pulse_nb, FloatText_decaytime_sipm, CheckBox_Gaussian_filter_sipm, 
             FloatText_Gaussian_smooth_sipm, CheckBox_Baseline_removed_sipm, CheckBox_PSD_sipm])

col2 = VBox([IntText_Nb_boards, IntText_Nb_channel_board, CheckBox_Charge, CheckBox_all_charge_pulses, 
             IntText_Pulse_nb_charge, FloatText_decaytime_charge, CheckBox_Gaussian_filter_charge, 
             FloatText_Gaussian_smooth_charge, CheckBox_Baseline_removed_charge, CheckBox_Processed_pulse_charge,
             CheckBox_PSD_charge])


display(HBox([col1, col2]), output)






Output()

In [11]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this notebook is hidden by default for readability purposes.
If you would like to see the raw code, click <a href="javascript:code_toggle()">here</a>.''')